<a href="https://colab.research.google.com/github/kostyav/colab/blob/master/sg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hello Baptiste, Fabien

This is the demo version of news summarization 

Pls pick the dates and the query (in the search string)

In [81]:
#@title Dates and search strings { run: "auto", vertical-output: true, display-mode: "form" }
begin = '2018-11-10' #@param {type:"date"}
end = '2018-11-13' #@param {type:"date"}
search = 'Takeda' #@param {type:"string"}

res=run_search(begin, end, search)


display(HTML(res.to_html()))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [0]:
!pip install langdetect python-Levenshtein fuzzywuzzy 
!python -m spacy download en

In [0]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from langdetect import detect, DetectorFactory
import spacy
import re
from fuzzywuzzy import fuzz
from itertools import combinations,groupby,chain
from IPython.display import HTML, display

pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

nlp = spacy.load('en')

In [0]:
def run_search(beginDate, endDate, query):
  bb = beginDate.replace('-','')+'000000'
  ee = endDate.replace('-','')+'999999'
  ts = beginDate+' 00:00:00'
  
  bqq="SELECT GKGRECORDID, DATE,DocumentIdentifier FROM  `gdelt-bq.gdeltv2.gkg_partitioned`"+\
  " WHERE REGEXP_CONTAINS(V2Organizations, r'(?i).*"+query+".*') AND DATE<="+ee+\
  " AND DATE>="+bb+" AND _PARTITIONTIME >= TIMESTAMP(\""+ts+"\")"
  
  project_id='gdet-socgen'
  
  df = pd.io.gbq.read_gbq(bqq, project_id=project_id, 
                        verbose=False, dialect='standard')
  
  df.DATE=pd.to_datetime(df.DATE,format='%Y%m%d%H%M%S')
  df=df.sort_values(by=['DATE'],ascending=False, inplace=False)
  
  dd=[(t[0],t[1],t[2]) for t in df.values.tolist()]
  
  o=list(analyze_links(dd))
  
  idxs=[(a,b) for a,b in combinations(range(len(o)),2) if fuzz.ratio(o[a][2][1],o[b][2][1])>80]
  gg=dict({i:set(map(lambda t:t[1], g)) for i,g in  groupby(idxs,lambda t:t[0])})
  idx2remove=set(chain.from_iterable(gg.values()))
  clean=[o[oo] for oo in range(len(o)) if oo not in idx2remove]
  
  res = pd.DataFrame.from_records([(c[1], c[2][0], c[2][1], c[2][2]) for c in clean], 
                                  columns=['Date','url','summary','header'])
  
  return res

In [0]:
def get_surrounding_sents(txt):
    doc = nlp(txt)
    ss=[s.text for s in doc.sents]
    sent_window=2
    ind_winds=[(max(0,i-sent_window), min(len(ss), i+sent_window)) 
               for (i,s) in enumerate(ss) if re.match(r'.*takeda.*', s, re.I)]
    if ind_winds:
        lastt=ind_winds[0]
        ind_winds2=[]
        for t in ind_winds:
            if (t[0]<=lastt[1]):
                lastt=(lastt[0], t[1])
            else:
                ind_winds2.append(lastt)
                lastt=t
        ind_winds2.append(lastt)
        txt4sum=' '.join(chain.from_iterable([ss[start:end] for (start,end) in ind_winds2]))
        return txt4sum
    else:
        return None

def get_txt(url):
    text, h = ('','')
    try:
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html)
        h = soup(['h1'])[0].contents[0]
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text()
    except Exception as exc:
        return None
    else:
        sumr = get_surrounding_sents(text)
        l=detect(text)
        if l == 'en' and sumr != None:
            return (url, sumr, h)
        else:
            return None

def analyze_links(dd):
    for l in dd:
        r=get_txt(l[2])
        if r!=None:
            yield (l[0], l[1], r)